In [1]:
import os
import tweepy

def credentials_dict() -> dict:
    return {
        "consumer_key": os.getenv("twitter_consumer_api_key"),
        "consumer_secret": os.getenv("twitter_consumer_api_secret"),
        "access_token": os.getenv("twitter_access_api_key"),
        "access_token_secret": os.getenv("twitter_access_api_secret")
    }
credentials = credentials_dict()

In [2]:
auth = tweepy.OAuthHandler(credentials["consumer_key"], credentials["consumer_secret"])
auth.set_access_token(credentials["access_token"], credentials["access_token_secret"])
api = tweepy.API(auth)
available_loc = api.available_trends()

In [3]:
available_loc = api.available_trends()

In [4]:
streaming_client = tweepy.StreamingClient(os.environ.get("bearer_token"))
bb = streaming_client.sample()

In [1]:
### v2
from datetime import datetime, timedelta
import os

import tweepy
import pandas as pd
import requests

yesterday_ts = pd.to_datetime(datetime.now()).floor('H') - timedelta(days = 1)

client = tweepy.Client( bearer_token = os.getenv("bearer_token"), 
                        consumer_key = os.getenv("twitter_consumer_api_key"), 
                        consumer_secret = os.getenv("twitter_consumer_api_secret"), 
                        access_token = os.getenv("twitter_access_api_key"), 
                        access_token_secret = os.getenv("twitter_access_api_secret"), 
                        return_type = requests.Response,
                        wait_on_rate_limit = True)

In [22]:
# Define query
query = 'nba'

# get max. 100 tweets - , 'organic_metrics' errors out for me
df = client.search_recent_tweets(query=query, 
                                    tweet_fields=['id', 'created_at', 'author_id', 'geo', 'lang', 'source', 'public_metrics'],
                                     max_results=100)
df = pd.json_normalize(df.json()['data']) 

BadRequest: 400 Bad Request
There were errors processing your request: Reference to invalid field 'filter' (at position 6), Reference to invalid operator 'filter'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 6)

In [15]:
# Save data as dictionary
tweets_dict = df.json() 

# # Extract "data" value from dictionary
# tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
# df = pd.json_normalize(tweets.json()['data']) 

In [44]:
query = "nba"

# Granularity is what you want the timeseries count data to be grouped by
# You can request minute, hour, or day granularity
# The default granularity, if not specified is hour

# it groups data by the hour, so yesterday_ts is the yesterday's timestamp rounded down to the nearest hour.
response = client.get_recent_tweets_count(query, granularity="hour", start_time = yesterday_ts)

df = pd.DataFrame(response.json()['data'])